In [1]:
!curl ifconfig.me

34.90.72.229

In [2]:
pip install --upgrade snowflake-connector-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 kB 13.0 MB/s eta 0:00:00


In [3]:
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta
import snowflake.connector
from snowflake.connector.pandas_tools import write_pandas

# Configuración de la conexión a Snowflake
def get_snowflake_connection():
    return snowflake.connector.connect(
        account='',
        user='',
        password='',
        warehouse='',
        database='',
        schema='',
        role=''
    )

def load_company_data():
    conn = get_snowflake_connection()
    cursor = conn.cursor()
    sp500 = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]
    sp500 = sp500[['Symbol', 'Security', 'GICS Sector', 'GICS Sub-Industry']]
    sp500.columns = ['TICKER', 'COMPANY_NAME', 'SECTOR', 'INDUSTRY']

    success, nchunks, nrows, _ = write_pandas(conn, sp500, 'DIM_COMPANY')
    print(f"Loaded {nrows} rows into DIM_COMPANY in {nchunks} chunks. Success: {success}")

    conn.commit()
    cursor.close()
    conn.close()

def load_date_data():
    conn = get_snowflake_connection()
    cursor = conn.cursor()
    end_date = datetime.now()
    start_date = end_date - timedelta(days=5*365)

    date_range = pd.date_range(start=start_date, end=end_date)
    date_df = pd.DataFrame({
        'DATE_ID': date_range,
        'YEAR': date_range.year,
        'MONTH': date_range.month,
        'DAY': date_range.day,
        'DAY_OF_WEEK': date_range.dayofweek,
        'IS_TRADING_DAY': date_range.dayofweek < 5
    })
    date_df['DATE_ID'] = date_df['DATE_ID'].dt.date

    success, nchunks, nrows, _ = write_pandas(conn, date_df, 'DIM_DATE')
    print(f"Loaded {nrows} rows into DIM_DATE in {nchunks} chunks. Success: {success}")

    conn.commit()
    cursor.close()
    conn.close()

def load_stock_price_data():
    conn = get_snowflake_connection()
    cursor = conn.cursor()
    sp500 = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]
    tickers = sp500['Symbol'].tolist()
    end_date = datetime.now()
    start_date = end_date - timedelta(days=5*365)

    for ticker in tickers:
        try:
            stock_data = yf.download(ticker, start=start_date, end=end_date)
            if stock_data.empty:
                print(f"No data available for {ticker}. Skipping...")
                continue

            stock_data.reset_index(inplace=True)
            stock_data['ticker'] = ticker
            stock_data = stock_data.rename(columns={
                'Date': 'DATE_ID',
                'Open': 'OPEN_PRICE',
                'High': 'HIGH_PRICE',
                'Low': 'LOW_PRICE',
                'Close': 'CLOSE_PRICE',
                'Volume': 'VOLUME'
            })

            stock_data['DATE_ID'] = pd.to_datetime(stock_data['DATE_ID'])

            cursor.execute(f"SELECT COMPANY_ID FROM DIM_COMPANY WHERE TICKER = '{ticker}'")
            result = cursor.fetchone()
            if result is None:
                print(f"No company_id found for ticker {ticker}. Skipping...")
                continue
            company_id = result[0]
            stock_data['COMPANY_ID'] = company_id

            stock_data = stock_data[['COMPANY_ID', 'DATE_ID', 'OPEN_PRICE', 'HIGH_PRICE', 'LOW_PRICE', 'CLOSE_PRICE', 'VOLUME']]
            stock_data['DATE_ID'] = stock_data['DATE_ID'].dt.date

            success, nchunks, nrows, _ = write_pandas(conn, stock_data, 'FACT_STOCK_PRICE')
            print(f"Loaded {nrows} rows for {ticker} into FACT_STOCK_PRICE in {nchunks} chunks. Success: {success}")

            conn.commit()

        except Exception as e:
            print(f"Error processing {ticker}: {str(e)}")
            continue

    cursor.close()
    conn.close()

# Ejecutar funciones
load_company_data()
load_date_data()
load_stock_price_data()

Loaded 503 rows into DIM_COMPANY in 1 chunks. Success: True
Loaded 1826 rows into DIM_DATE in 1 chunks. Success: True


[*********************100%%**********************]  1 of 1 completed


Loaded 1256 rows for MMM into FACT_STOCK_PRICE in 1 chunks. Success: True


[*********************100%%**********************]  1 of 1 completed


Loaded 1256 rows for AOS into FACT_STOCK_PRICE in 1 chunks. Success: True


...

Loaded 1256 rows for ZBRA into FACT_STOCK_PRICE in 1 chunks. Success: True


[*********************100%%**********************]  1 of 1 completed


Loaded 1256 rows for ZBH into FACT_STOCK_PRICE in 1 chunks. Success: True


[*********************100%%**********************]  1 of 1 completed


Loaded 1256 rows for ZTS into FACT_STOCK_PRICE in 1 chunks. Success: True
